In [ ]:
# On remet les bases système (ça ira vite cette fois)
!apt-get update && apt-get install -y libsndfile1 ffmpeg sox

# On relance l'installation de NeMo
print("⏳ Reprise de l'installation de NeMo...")
print("⚠️ ATTENTION : L'étape 'Building wheel for sox' peut durer 3 minutes. NE TOUCHEZ À RIEN !")

!pip install "nemo_toolkit[asr] @ git+https://github.com/NVIDIA/NeMo.git@r1.23.0"

print("\n✅ C'est (vraiment) fini cette fois.")

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,341 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:13 http://security.ubuntu.com/ub

In [1]:
# 1. On nettoie les fauteurs de trouble
!pip uninstall -y huggingface_hub transformers

# 2. On installe le "Trio Magique" compatible avec NeMo 1.23
# huggingface_hub 0.23.2 contient A LA FOIS 'HfFolder' et 'is_offline_mode'
!pip install "huggingface_hub==0.23.2" "transformers==4.38.2" "numpy<2.0"

print("✅ Versions alignées.")

Found existing installation: huggingface_hub 0.36.2
Uninstalling huggingface_hub-0.36.2:
  Successfully uninstalled huggingface_hub-0.36.2
Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
  Using cached huggingface_hub-0.23.2-py3-none-any.whl.metadata (12 kB)
  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached huggingface_hub-0.23.2-py3-none-any.whl (401 kB)
Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

✅ Versions alignées.


In [1]:
import nemo
import nemo.collections.asr as nemo_asr
print("🎉 NeMo est opérationnel (Enfin) !")

[NeMo W 2026-02-08 06:51:27 nemo_logging:393] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
      m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
    
[NeMo W 2026-02-08 06:51:27 nemo_logging:393] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
      m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
    
[NeMo W 2026-02-08 06:51:27 nemo_logging:393] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
      elif re.match('(flt)p?( \(default\))?$', token):
    
[NeMo W 2026-02-08 06:51:27 nemo_logging:393] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
      elif re.match('(dbl)p?( \(default\))?$', token):
    


🎉 NeMo est opérationnel (Enfin) !


In [2]:
import os
import json
import wget
import glob
import librosa
import csv
from omegaconf import OmegaConf

# ================= 🚑 PATCH ANTI-CRASH NEMO =================
import nemo.core.utils.numba_utils as numba_utils
numba_utils.numba_cuda_is_supported = lambda *args: False

# Imports pour l'évaluation (On utilise les métriques Pyannote pour juger NeMo)
from pyannote.core import Annotation, Segment
from pyannote.metrics.diarization import DiarizationErrorRate
from nemo.collections.asr.models import ClusteringDiarizer

# ================= CONFIGURATION =================
# Chemins d'entrée (Les mêmes que pour Pyannote)
AUDIO_DIR = "/content/drive/MyDrive/GroundTruth_overlap_noise/audioMono"
GT_RTTM_DIR = "/content/drive/MyDrive/rttm"

# Dossiers de sortie pour NeMo
OUTPUT_DIR = "/content/drive/MyDrive/nemo_overlap_noise_resultaaaa"
PRED_RTTM_DIR = os.path.join(OUTPUT_DIR, "pred_rttms") # Là où NeMo va écrire
CSV_OUTPUT_PATH = "/content/drive/MyDrive/GroundTruth_overlap_noise/resultats_nemo250ms.csv"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ================= 1. PRÉPARATION DU MANIFESTE =================
manifest_path = os.path.join(OUTPUT_DIR, "input_manifest.json")
audio_files = glob.glob(os.path.join(AUDIO_DIR, "*.wav"))

if not audio_files:
    print("❌ ERREUR : Aucun fichier .wav trouvé !")
    exit()

print(f"📝 Génération du manifeste NeMo pour {len(audio_files)} fichiers...")

with open(manifest_path, 'w') as fout:
    for audio_file in audio_files:
        try:
            duration = librosa.get_duration(path=audio_file)
            meta = {
                "audio_filepath": audio_file,
                "offset": 0,
                "duration": duration,
                "label": "infer",
                "text": "-",
                "num_speakers": None,
                "rttm_filepath": None,
                "uem_filepath": None
            }
            json.dump(meta, fout)
            fout.write('\n')
        except Exception as e:
            print(f"⚠️ Erreur fichier {os.path.basename(audio_file)}: {e}")

# ================= 2. CONFIGURATION & INFÉRENCE NEMO =================
config_url = "https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/diarization/conf/inference/diar_infer_telephonic.yaml"
config_path = os.path.join(OUTPUT_DIR, "diar_infer_telephonic.yaml")

if not os.path.exists(config_path):
    wget.download(config_url, config_path)

config = OmegaConf.load(config_path)
config.diarizer.manifest_filepath = manifest_path
config.diarizer.out_dir = OUTPUT_DIR
config.diarizer.speaker_embeddings.model_path = "titanet_large" # Modèle robuste

# Réglages VAD et Clustering
config.diarizer.oracle_vad = False # On laisse NeMo détecter la voix
config.diarizer.clustering.parameters.oracle_num_speakers = False # On laisse NeMo deviner le nombre de speakers

print("\n🚀 Démarrage de l'inférence NeMo (Cela peut prendre du temps)...")
try:
    sd_model = ClusteringDiarizer(cfg=config)
    sd_model.diarize()
    print("✅ Inférence NeMo terminée.")
except Exception as e:
    print(f"❌ Erreur critique NeMo : {e}")
    exit()

# ================= 3. ÉVALUATION (COMPARAISON AVEC GROUND TRUTH) =================
print("\n🔍 Démarrage de l'évaluation comparative...")

# Fonction pour charger un RTTM (utilisée pour GT et pour Prédiction NeMo)
def load_rttm(file_path):
    annotation = Annotation()
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 8 and parts[0] == 'SPEAKER':
                start = float(parts[3])
                duration = float(parts[4])
                label = parts[7]
                annotation[Segment(start, start + duration)] = label
    return annotation

# Métrique (Mêmes paramètres que pour Pyannote)
der_metric = DiarizationErrorRate(collar=0.25, skip_overlap=False) # Mettez collar=0.25 si vous voulez être plus tolérant
results_data = []

print(f"{'FICHIER':<30} | {'DER (%)':<10} | {'VRAI':<5} | {'PRED':<5} | {'DIFF':<5}")
print("-" * 80)

for audio_file in audio_files:
    filename = os.path.basename(audio_file)
    file_id = os.path.splitext(filename)[0]

    # Chemins
    gt_path = os.path.join(GT_RTTM_DIR, file_id + ".rttm")
    pred_path = os.path.join(PRED_RTTM_DIR, file_id + ".rttm") # NeMo génère ici

    if not os.path.exists(gt_path):
        print(f"⚠️ GT manquant pour {filename}")
        continue

    if not os.path.exists(pred_path):
        print(f"⚠️ Prédiction NeMo manquante pour {filename} (VAD vide ?)")
        # On peut compter ça comme 100% d'erreur ou ignorer
        continue

    try:
        # A. Chargement
        reference = load_rttm(gt_path)
        hypothesis = load_rttm(pred_path) # On charge le fichier généré par NeMo

        # B. Calcul DER
        # UEM basé sur la durée réelle du fichier audio (plus précis)
        audio_duration = librosa.get_duration(path=audio_file)
        uem = Segment(0, audio_duration)

        der_val = der_metric(reference, hypothesis, uem=uem) * 100

        # C. Comptage
        true_speakers = len(reference.labels())
        pred_speakers = len(hypothesis.labels())
        diff = pred_speakers - true_speakers

        results_data.append({
            "filename": filename,
            "der": der_val,
            "true_speakers": true_speakers,
            "pred_speakers": pred_speakers,
            "diff": diff
        })

        print(f"{filename:<30} | {der_val:.2f}%      | {true_speakers:<5} | {pred_speakers:<5} | {diff:+d}")

    except Exception as e:
        print(f"❌ Erreur évaluation {filename}: {e}")

# ================= 4. STATISTIQUES & CSV =================
if results_data:
    all_ders = [r['der'] for r in results_data]
    min_der = min(all_ders)
    max_der = max(all_ders)
    best_file = next(r['filename'] for r in results_data if r['der'] == min_der)
    worst_file = next(r['filename'] for r in results_data if r['der'] == max_der)

    global_der = abs(der_metric) * 100
    correct_counts = sum(1 for r in results_data if r['diff'] == 0)
    accuracy_count = (correct_counts / len(results_data)) * 100

    print("\n" + "="*40)
    print(" RÉSULTATS GLOBAUX NEMO ")
    print("="*40)
    print(f"DER Global (Moyenne Pondérée) : {global_der:.2f}%")
    print(f"DER Minimum (Meilleur)        : {min_der:.2f}% ({best_file})")
    print(f"DER Maximum (Pire)            : {max_der:.2f}% ({worst_file})")
    print(f"Précision Nb Locuteurs        : {accuracy_count:.2f}%")

    print(f"\nSauvegarde CSV dans : {CSV_OUTPUT_PATH}")
    with open(CSV_OUTPUT_PATH, mode='w', newline='') as csv_file:
        fieldnames = ['Filename', 'DER', 'True_Speakers', 'Pred_Speakers', 'Diff']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for data in results_data:
            writer.writerow({
                'Filename': data['filename'],
                'DER': f"{data['der']:.4f}",
                'True_Speakers': data['true_speakers'],
                'Pred_Speakers': data['pred_speakers'],
                'Diff': data['diff']
            })
    print("✅ CSV généré.")
else:
    print("Aucun résultat à analyser.")

📝 Génération du manifeste NeMo pour 30 fichiers...

🚀 Démarrage de l'inférence NeMo (Cela peut prendre du temps)...
[NeMo I 2026-02-08 06:53:03 nemo_logging:381] Loading pretrained vad_multilingual_marblenet model from NGC
[NeMo I 2026-02-08 06:53:03 nemo_logging:381] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/vad_multilingual_marblenet/versions/1.10.0/files/vad_multilingual_marblenet.nemo to /root/.cache/torch/NeMo/NeMo_1.23.0/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo
[NeMo I 2026-02-08 06:53:03 nemo_logging:381] Instantiating model from pre-trained checkpoint


[NeMo W 2026-02-08 06:53:03 nemo_logging:393] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/ami_train_0.63.json,/manifests/freesound_background_train.json,/manifests/freesound_laughter_train.json,/manifests/fisher_2004_background.json,/manifests/fisher_2004_speech_sampled.json,/manifests/google_train_manifest.json,/manifests/icsi_all_0.63.json,/manifests/musan_freesound_train.json,/manifests/musan_music_train.json,/manifests/musan_soundbible_train.json,/manifests/mandarin_train_sample.json,/manifests/german_train_sample.json,/manifests/spanish_train_sample.json,/manifests/french_train_sample.json,/manifests/russian_train_sample.json
    sample_rate: 16000
    labels:
    - background
    - speech
    batch_size: 256
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy

[NeMo I 2026-02-08 06:53:03 nemo_logging:381] PADDING: 16
[NeMo I 2026-02-08 06:53:03 nemo_logging:381] Model EncDecClassificationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.23.0/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo.
[NeMo I 2026-02-08 06:53:03 nemo_logging:381] Loading pretrained titanet_large model from NGC
[NeMo I 2026-02-08 06:53:03 nemo_logging:381] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/titanet_large/versions/v1/files/titanet-l.nemo to /root/.cache/torch/NeMo/NeMo_1.23.0/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo
[NeMo I 2026-02-08 06:53:04 nemo_logging:381] Instantiating model from pre-trained checkpoint


[NeMo W 2026-02-08 06:53:04 nemo_logging:393] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2026-02-08 06:53:04 nemo_logging:393] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data

[NeMo I 2026-02-08 06:53:04 nemo_logging:381] PADDING: 16
[NeMo I 2026-02-08 06:53:05 nemo_logging:381] Model EncDecSpeakerLabelModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.23.0/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo.
[NeMo I 2026-02-08 06:53:05 nemo_logging:381] Number of files to diarize: 30
[NeMo I 2026-02-08 06:53:05 nemo_logging:381] Split long audio file to avoid CUDA memory issue


...

[NeMo I 2026-02-08 06:54:10 nemo_logging:381] Saved embedding files to /content/drive/MyDrive/nemo_overlap_noise_resultaaaa/speaker_outputs/embeddings



clustering: 100%|██████████| 30/30 [00:06<00:00,  4.51it/s]

[NeMo I 2026-02-08 06:54:16 nemo_logging:381] Outputs are saved in /content/drive/MyDrive/nemo_overlap_noise_resultaaaa directory



[NeMo W 2026-02-08 06:54:16 nemo_logging:393] Check if each ground truth RTTMs were present in the provided manifest file. Skipping calculation of Diariazation Error Rate


✅ Inférence NeMo terminée.

🔍 Démarrage de l'évaluation comparative...
FICHIER                        | DER (%)    | VRAI  | PRED  | DIFF 
--------------------------------------------------------------------------------
hard_overlap_005.wav           | 69.07%      | 2     | 1     | -1
hard_overlap_008.wav           | 41.63%      | 2     | 2     | +0
hard_overlap_027.wav           | 34.52%      | 2     | 2     | +0
hard_overlap_013.wav           | 31.92%      | 2     | 2     | +0
hard_overlap_001.wav           | 56.73%      | 2     | 1     | -1
hard_overlap_017.wav           | 29.21%      | 2     | 2     | +0
hard_overlap_000.wav           | 72.91%      | 2     | 1     | -1
hard_overlap_023.wav           | 74.90%      | 2     | 1     | -1
hard_overlap_003.wav           | 36.06%      | 2     | 2     | +0
hard_overlap_029.wav           | 38.33%      | 2     | 1     | -1
hard_overlap_007.wav           | 27.41%      | 2     | 2     | +0
hard_overlap_019.wav           | 37.04%      | 2     |